In [48]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [2]:
os.environ['OPENAI_API_KEY'] = '<YOUR_OPENAI_API_KEY>'
os.environ['PINECONE_API_KEY'] = '<YOUR_PINECONE_API_KEY>'

In [3]:
index_name = "test"
embeddings = OpenAIEmbeddings()

In [17]:
 # path to an example text file
loader = TextLoader("./text_data/data.txt",encoding = 'UTF-8')

In [18]:
documents = loader.load()

In [27]:
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"])

In [28]:
docs = text_splitter.split_documents(documents)

In [33]:
# for i in docs:
#     print(i.page_content)
#     print("-------------------------------")

In [34]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        docs,
        index_name=index_name,
        embedding=embeddings
    )

In [43]:
vectorstore_from_docs.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001B72D3F8990>)

In [46]:
llm = OpenAI()

C:\Users\Vikas\anaconda3\envs\GenAI\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [49]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore_from_docs.as_retriever())

In [51]:
qa.run("who is Thomas Brown Jr.")

C:\Users\Vikas\anaconda3\envs\GenAI\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' Thomas Brown Jr. is the uncle of Ketanji Brown Jackson, a judge and current nominee for the Supreme Court of the United States. He was sentenced to life in prison for a nonviolent cocaine conviction in 1989.'

In [39]:
# query = "in which year, Thomas Brown Jr. was sentenced to life in prison?"
# res = vectorstore_from_docs.similarity_search(query)

In [52]:
# for i in res:
#     print(i.page_content)
#     print("|||||||||||||||||||||||||||||||||")
#     print(i.metadata)
#     print("-------------------------------")

# code for system to ask query from user and revert from vectorDB

In [ ]:
import sys
while True:
    user_input = input(f"Input Prompt: ")
    if user_input == 'exit':
        print("Exiting")
        sys.exit()
    if user_input == '':
        continue
    result = qa({'query':user_input})
    print(f"Answer: {result['result']}")

Input Prompt:  who is Ketanji Brown Jackson?


Answer:  Ketanji Brown Jackson is an American lawyer and jurist who is currently an Associate Justice of the Supreme Court of the United States. She was nominated by President Joe Biden and assumed office on June 30, 2022. Prior to this, she served as a judge on the United States Court of Appeals for the District of Columbia Circuit and the United States District Court for the District of Columbia. She was also the Vice Chair of the United States Sentencing Commission from 2010 to 2014. 


Input Prompt:  when she graduate?


Answer:  She graduated from Harvard in 1992 with a Bachelor of Arts, magna cum laude.
